# Clustering Crypto

In [24]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [25]:
# Load the crypto_data.csv dataset.
file_path = 'crypto_data.csv'
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [26]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df['IsTrading']==True]
print(crypto_df.shape)
crypto_df.head()

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [27]:
# Keep all the cryptocurrencies that have a working algorithm.
# YOUR CODE HERE

In [28]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns='IsTrading', inplace=True)
print(crypto_df.shape)
crypto_df.head()

(1144, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [29]:
# Remove rows that have at least 1 null value.
crypto_df.dropna(inplace=True)
print(crypto_df.shape)
crypto_df.head()

(685, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [30]:
crypto_df['TotalCoinsMined'].value_counts()

0.000000e+00    152
1.000000e+09      7
1.000000e+08      7
2.500000e+07      2
2.000000e+07      2
               ... 
2.951355e+08      1
3.082571e+10      1
2.108935e+07      1
1.207310e+06      1
7.847417e+06      1
Name: TotalCoinsMined, Length: 520, dtype: int64

In [31]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df['TotalCoinsMined']>0]
print(crypto_df.shape)
crypto_df.head()

(532, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [32]:
# Create a new DataFrame that holds only the cryptocurrencies names
coin_df = crypto_df[['CoinName']]
print(coin_df.shape)
coin_df.head()

(532, 1)


,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [33]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns='CoinName', inplace=True)
print(crypto_df.shape)
crypto_df.head()

(532, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [34]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
print(X.shape)
X.head()

(532, 98)


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# Standardize the data with StandardScaler().
data_scaler = StandardScaler()
X_scaled = data_scaler.fit_transform(X)
X_scaled[:5]

array([[-0.11710817, -0.1528703 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.06142951, -0.07530656, -0.0433963 , -0.06142951, -0.06142951,
        -0.0433963 , -0.0433963 , -0.19245009, -0.06142951, -0.09740465,
        -0.0433963 , -0.11547005, -0.07530656, -0.0433963 , -0.0433963 ,
        -0.15191091, -0.0433963 , -0.13118084, -0.0433963 , -0.0433963 ,
        -0.08703883, -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.06142951, -0.0433963 , -0.08703883, -0.08703883, -0.08703883,
        -0.0433963 , -0.13118084, -0.13840913, -0.13840913, -0.0433963 ,
        -0.06142951, -0.0433963 , -0.07530656, -0.18168574, -0.0433963 ,
        -0.0433963 , -0.0433963 , -0.07530656, -0.15826614, -0.31491833,
        -0.0433963 , -0.08703883, -0.07530656, -0.06142951,  1.38675049,
        -0.0433963 , -0.0433963 , -0.06142951, -0.0433963 , -0.0433963 ,
        -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.0433963 , -0.39879994, -0.0433963 , -0.1

### Deliverable 2: Reducing Data Dimensions Using PCA

In [36]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)

In [37]:
# Create a DataFrame with the three principal components.
X_pca = pca.fit_transform(X_scaled)
pcs_df = pd.DataFrame(
    data=X_pca, columns=['PC1', 'PC2', 'PC3'], index=X.index
)
pcs_df.head()

,PC1,PC2,PC3
42,-0.338146,1.071857,-0.426432
404,-0.321499,1.071983,-0.426509
1337,2.292047,1.658951,-0.392447
BTC,-0.152803,-1.310102,0.178866
ETH,-0.148274,-2.040160,0.265101


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [38]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(X_scaled)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [39]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(X)

# Predict clusters
predictions = model.predict(X)

In [52]:
print(pcs_df.shape)

(532, 3)


In [55]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df], axis=1, join='inner')

# Add the predicted class columns
clustered_df["class"] = model.labels_


#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = coin_df

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.338146,1.071857,-0.426432,0,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.321499,1.071983,-0.426509,0,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.292047,1.658951,-0.392447,0,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.152803,-1.310102,0.178866,1,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.148274,-2.040160,0.265101,1,Ethereum
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.160199,-1.098246,-0.045338,1,Litecoin
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.409285,1.203804,-0.310373,0,Dash
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.144910,-2.252604,0.406587,1,Monero
ETC,Ethash,PoW,1.133597e+08,210000000,-0.146717,-2.040269,0.265100,1,Ethereum Classic
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.170128,-1.979673,0.456434,1,ZCash


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [60]:
# Creating a 3D-Scatter with the PCA data and the clusters
import plotly.express as px
fig = px.scatter_3d(
     clustered_df,
     x="PC3",
     y="PC2",
     z="PC1",
     color="class",
     hover_name="CoinName", 
     hover_data=["Algorithm"])

fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [62]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(sortable=True, selectable=True)


:Table   [Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,class,CoinName]

In [64]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE
total = clustered_df['CoinName'].count()

print(f'There are {total} tradable cryptocurrencies.')

There are 532 tradable cryptocurrencies.


In [73]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
clustered_df[['TotalCoinSupply', 'TotalCoinsMined']] = MinMaxScaler().fit_transform(clustered_df[['TotalCoinSupply', 'TotalCoinsMined']])
clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,class,CoinName
42,Scrypt,PoW/PoS,0.000000e+00,4.200000e-11,-0.338146,1.071857,-0.426432,0,42 Coin
404,Scrypt,PoW/PoS,1.065855e-03,5.320000e-04,-0.321499,1.071983,-0.426509,0,404Coin
1337,X13,PoW/PoS,2.957551e-02,3.141593e-01,2.292047,1.658951,-0.392447,0,EliteCoin
BTC,SHA-256,PoW,1.810842e-05,2.100000e-05,-0.152803,-1.310102,0.178866,1,Bitcoin
ETH,Ethash,PoW,1.087731e-04,0.000000e+00,-0.148274,-2.040160,0.265101,1,Ethereum
...,...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.020225e-03,2.000000e-03,2.471901,0.898048,-0.047953,0,ZEPHYR
GAP,Scrypt,PoW/PoS,1.508199e-05,2.500000e-04,-0.336193,1.071731,-0.426435,0,Gapcoin
BDX,CryptoNight,PoW,9.901351e-04,1.400223e-03,0.327491,-2.342049,0.265557,1,Beldex
ZEN,Equihash,PoW,7.370282e-06,2.100000e-05,-0.170129,-1.979674,0.456434,1,Horizen


In [74]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = clustered_df


plot_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,class,CoinName
42,Scrypt,PoW/PoS,0.000000,4.200000e-11,-0.338146,1.071857,-0.426432,0,42 Coin
404,Scrypt,PoW/PoS,0.001066,5.320000e-04,-0.321499,1.071983,-0.426509,0,404Coin
1337,X13,PoW/PoS,0.029576,3.141593e-01,2.292047,1.658951,-0.392447,0,EliteCoin
BTC,SHA-256,PoW,0.000018,2.100000e-05,-0.152803,-1.310102,0.178866,1,Bitcoin
ETH,Ethash,PoW,0.000109,0.000000e+00,-0.148274,-2.040160,0.265101,1,Ethereum
LTC,Scrypt,PoW,0.000064,8.400000e-05,-0.160199,-1.098246,-0.045338,1,Litecoin
DASH,X11,PoW/PoS,0.000009,2.200000e-05,-0.409285,1.203804,-0.310373,0,Dash
XMR,CryptoNight-V7,PoW,0.000017,0.000000e+00,-0.144910,-2.252604,0.406587,1,Monero
ETC,Ethash,PoW,0.000115,2.100000e-04,-0.146717,-2.040269,0.265100,1,Ethereum Classic
ZEC,Equihash,PoW,0.000007,2.100000e-05,-0.170128,-1.979673,0.456434,1,ZCash


In [80]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y=["TotalCoinSupply"], by="class", value_label="CoinName")


:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply)